## Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [44]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import anthropic as ant
import google.generativeai as genai

In [45]:
load_dotenv(override=True)

API_KEYS = {
    "OpenAI": ("OPENAI_API_KEY", 8),
    "Anthropic": ("ANTHROPIC_API_KEY", 7),
    "Google": ("GOOGLE_API_KEY", 2)
}

keys = {label: os.getenv(env) for label, (env, _) in API_KEYS.items()}

def check_key(label, prefix_len):
    key = keys[label]
    if key:
        print(f"{label} API Key exists and begins {key[:prefix_len]}")
    else:
        print(f"{label} API Key not set (optional)")

for label, (env_name, prefix_len) in API_KEYS.items():
    check_key(label, prefix_len)


OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI


In [46]:
openai  = OpenAI(api_key=keys["OpenAI"])
anthropic = ant.Anthropic(api_key=keys["Anthropic"])
genai.configure(api_key=keys["Google"])

In [47]:
max_tokens = 512
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "models/gemini-2.0-flash"
system_prompt = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [57]:
def chat_gpt(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=gpt_model, messages=messages)
    yield response.choices[0].message.content

In [58]:
def chat_claude(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = history + [{"role": "user", "content": message}]
    response = anthropic.messages.create(
        model=claude_model,
        system=system_prompt,
        messages=messages,
        max_tokens=max_tokens
    )
    yield response.content[0].text

In [59]:
def chat_gemini(message, history):
    gemini_history = []
    
    # 1. Convert history from OpenAI format to Gemini format
    for h in history:
        # Map 'assistant' to 'model', 'user' stays the same
        role = "model" if h["role"] == "assistant" else "user"
        
        # Ignore system messages
        if h["role"] == "system":
            continue
            
        content = h["content"]
        
        # If 'content' is a list (Gradio multimodal format), we extract only the text
        if isinstance(content, list):
            # We join all the parts that are text.
            content = " ".join([part["text"] for part in content if "text" in part])
        
        # Now 'content' is safely a string
        gemini_history.append({
            "role": role,
            "parts": [content]
        })
    
    # 2. Add the user's current message
    gemini_history.append({
        "role": "user",
        "parts": [message]
    })

    config = genai.types.GenerationConfig(
        max_output_tokens=max_tokens,
        temperature=0.1
    )

    model = genai.GenerativeModel(
        model_name=gemini_model,
        system_instruction=system_prompt
    )

    # 3. Transfer the complete list
    response = model.generate_content(gemini_history, generation_config=config)
    
    yield response.text

In [56]:
gr.ChatInterface(fn=chat_claude).launch()

* Running on local URL:  http://127.0.0.1:7900
* To create a public link, set `share=True` in `launch()`.


In [40]:
def chat_manager(message, history, model_name):
    
    if model_name == "Gemini":
        yield from chat_gemini(message, history)
        
    elif model_name == "GPT-4o":
        yield from chat_gpt(message, history)
        
    elif model_name == "Claude 3":
        yield from chat_claude(message, history)

In [51]:
model_dropdown = gr.Dropdown(
    choices=["Gemini", "GPT-4o", "Claude 3"], 
    value="Gemini", 
    label="Select the Model"
)

gr.ChatInterface(
    fn=chat_manager,
    additional_inputs=[model_dropdown],
    title="🤖 Multi-Model Chat",
    description="Chat with different AIs by changing the selector below."
).launch()

* Running on local URL:  http://127.0.0.1:7898
* To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [77]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"

In [66]:
get_ticket_price("London")

'The price of a ticket to London is $799'

In [67]:
# There's a particular dictionary structure that's required to describe our function:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [68]:
# And this is included in a list of tools:
tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [69]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [75]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=gpt_model, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=gpt_model, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7904
* To create a public link, set `share=True` in `launch()`.


Tool called for city London
Tool called for city Tokyo
